<a href="https://colab.research.google.com/github/nickdalfarra/4X_QLearning/blob/claire/CheckingStationarityWithADF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import io, base64, os, json, re
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime
from random import randint
from google.colab import files
import statsmodels
from statsmodels.tsa.stattools import adfuller

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
file = files.upload()

Saving USD_CAD.csv to USD_CAD.csv
Saving USDCAD_1min_2019.csv to USDCAD_1min_2019.csv


In [0]:
# load market data from Yahoo Finance (https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC)
#get data from my computer 
minute = pd.read_csv(io.BytesIO(file['USDCAD_1min_2019.csv']))
daily = pd.read_csv(io.BytesIO(file['USD_CAD.csv']))
#USDCAD_df['Date'] = pd.to_datetime(USDCAD_df['Date']+'  ' + USDCAD['Time'] +':00')
#USDCAD_DF['Time'] = pd.to_datetime(USDCAD_df['Time']+':00','%H:%M:%S')
#USDCAD_df.drop(['Time', 'Unknown'], axis=1)
#cut_off_date = '2010-01-01' 
daily = daily.head(261)
minute.head()

,Date,Time,Open,High,Low,Close,Unknown
0,2019.01.01,17:06,1.36234,1.36348,1.36208,1.36208,0
1,2019.01.01,17:07,1.36207,1.36229,1.36207,1.36228,0
2,2019.01.01,17:08,1.36228,1.36228,1.36228,1.36228,0
3,2019.01.01,17:09,1.36228,1.36256,1.36220,1.36256,0
4,2019.01.01,17:10,1.36256,1.36265,1.36256,1.36260,0


In [0]:
daily_gap = daily['Price'].pct_change()
minute_gap = minute['Close'].pct_change()
daily_gap = daily_gap.dropna()
minute_gap = minute_gap.dropna()
daily_price = daily['Price']
minute_price = minute['Close']
daily_gap.head()

1    0.000077
2    0.006006
3    0.000918
4    0.003441
5    0.002819
Name: Price, dtype: float64

In [0]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
    def ADF_Stationarity_Test(self, timeseries, printResults = True):
        #Dickey-Fuller test:
        adfTest = adfuller(timeseries, autolag='AIC')
        
        self.pValue = adfTest[1]
        
        if (self.pValue<self.SignificanceLevel):
            self.isStationary = True
        else:
            self.isStationary = False
        
        if printResults:
            dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adfTest[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)

In [0]:
sTest = StationarityTests()
sTest.ADF_Stationarity_Test(daily_price, printResults = True) ## daily data for a year 
print("Is the time series stationary? {0}".format(sTest.isStationary))

Augmented Dickey-Fuller Test Results:
ADF Test Statistic       -2.789136
P-Value                   0.059842
# Lags Used               0.000000
# Observations Used     260.000000
Critical Value (1%)      -3.455754
Critical Value (5%)      -2.872721
Critical Value (10%)     -2.572728
dtype: float64
Is the time series stationary? False
